In [2]:
# importing packages
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,OrdinalEncoder
from collections import defaultdict
labelEncoder_dict = defaultdict(LabelEncoder)

# from sklearn.preprocessing import StandardScaler

In [3]:
# Loading file data
df = pd.read_csv("survey_results_public.csv")
X = pd.DataFrame()

In [4]:
X['OpenSource'] = df['OpenSource'].eq('Yes').mul(1)
X['Hobby'] = df['Hobby'].eq('Yes').mul(1)
X['Student'] = df['Student'].str.contains('Yes').mul(1)


In [5]:
def CustomLabelEncoder(data):
    le = LabelEncoder()
    labels = le.fit_transform(data)
    mappings = {index: label for index, label in enumerate(fle.classes_)}
    return le,labels


### Label Encoding

In [6]:

# Categorical Encoding Columns
CategoricalColumns = ['Country','Employment','FormalEducation','UndergradMajor','CompanySize','YearsCoding']
for col in CategoricalColumns:
    X[col] = labelEncoder_dict[col].fit_transform(df[col])

In [7]:
# AssessJob and Benefits Added
df.fillna(df.iloc[:,17:38].mean(),inplace=True)
for col in df.iloc[:,17:38].columns:
    X[col] = df[col]



In [9]:
# JobSatisfaction Mapping
SatisfactionMapping = {
    'Extremely satisfied':6,
    'Moderately dissatisfied':5,
    'Moderately satisfied':4, 
    'Neither satisfied nor dissatisfied':3,
    'Slightly satisfied':2,
    'Slightly dissatisfied':1,
    'Extremely dissatisfied':0
}

df['JobSatisfaction'].replace(SatisfactionMapping,inplace=True)
df.fillna({'JobSatisfaction':3},inplace=True)
X['JobSatisfaction'] = df['JobSatisfaction']

# CareerSatisfaction
df['CareerSatisfaction'].replace(SatisfactionMapping,inplace=True)
df.fillna({'CareerSatisfaction':3},inplace=True)
X['CareerSatisfaction'] = df['CareerSatisfaction']



In [10]:
# HackathonReasons
X['HackathonParticipated'] = df['HackathonReasons'].notna()*1


In [11]:
# ConvertedSalary
X['ConvertedSalary'] = df['ConvertedSalary']


In [12]:
def CustomOneHotEncoding(data,X):
    temp = data.str.split(';', expand=True)
    new_columns = pd.unique(temp.values.ravel())
    for col in new_columns:
        X[col] = data.str.contains(col, regex=False).fillna(False)*1

In [13]:
# LanguageWorkedWith
CustomOneHotEncoding(df['LanguageWorkedWith'],X)


CustomOneHotEncoding(df['DevType'],X)

CustomOneHotEncoding(df['DatabaseWorkedWith'],X)

CustomOneHotEncoding(df['PlatformWorkedWith'],X)

CustomOneHotEncoding(df['FrameworkWorkedWith'],X)

CustomOneHotEncoding(df['IDE'],X)

CustomOneHotEncoding(df['Methodology'],X)

CustomOneHotEncoding(df['Methodology'],X)



In [15]:
# CheckInCode
CheckInCodeMapping = {
    'Multiple times per day':730, 
    'A few times per week':156, 
    'Weekly or a few times per month':52, 
    'Never':0,
    'Less than once per month':12, 
    'Once a day':365
}

X['CheckInCode'] = df['CheckInCode'].replace(CheckInCodeMapping)
X['CheckInCode'].fillna(X['CheckInCode'].mean(),inplace=True)

In [16]:
X

,OpenSource,Hobby,Student,Country,Employment,FormalEducation,UndergradMajor,CompanySize,YearsCoding,AssessJob1,...,Scrum,Kanban,Evidence-based software engineering,Formal standard such as ISO 9001 or IEEE 12207 (aka “waterfall” methodologies),Extreme programming (XP),Lean,Pair programming,Mob programming,PRINCE2,CheckInCode
0,0,1,0,85,1,1,10,4,7,10.000000,...,1,0,0,0,0,0,0,0,0,730.000000
1,1,1,0,173,0,1,3,2,8,1.000000,...,0,0,0,0,0,0,0,0,0,156.000000
2,1,1,0,175,0,0,6,4,5,6.397089,...,0,0,0,0,0,0,0,0,0,506.511547
3,0,0,0,175,0,1,6,3,3,6.397089,...,1,1,0,0,0,0,0,0,0,730.000000
4,0,1,1,150,0,8,6,2,9,8.000000,...,0,0,1,1,0,0,0,0,0,52.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98850,1,1,0,175,6,9,12,8,11,6.397089,...,0,0,0,0,0,0,0,0,0,506.511547
98851,1,0,0,152,4,9,12,8,0,6.397089,...,0,0,0,0,0,0,0,0,0,506.511547
98852,1,1,0,74,0,1,12,8,11,6.397089,...,0,0,0,0,0,0,0,0,0,506.511547
98853,0,1,0,137,2,8,12,8,11,6.397089,...,0,0,0,0,0,0,0,0,0,506.511547


In [ ]:
# # df['EducationTypes'].unique()

# x = .str.split(';').fillna('NA').tolist()
# devTypeSet = set()
# for devs in x:
#     try:
#         devTypeSet.update(devs)
#     except:
#         print(devs)

# list(devTypeSet)

In [ ]:
# # df['EducationTypes'].unique()

# x = df['SelfTaughtTypes'].str.split(';').fillna('NA').tolist()
# devTypeSet = set()
# for devs in x:
#     try:
#         devTypeSet.update(devs)
#     except:
#         print(devs)

# list(devTypeSet)